# RAG con SQL (Text-to-SQL)

Configuración inicial: carga de variables de entorno y cliente OpenAI.

In [ ]:
import sqlite3
import os
from pathlib import Path
import openai
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

API_HOST = os.getenv("API_HOST", "github")
client = openai.OpenAI(base_url="https://models.github.ai/inference", api_key=os.environ["GITHUB_TOKEN"])
MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

print(f"Using model: {MODEL_NAME}")

Conexión a la base de datos SQLite y extracción del esquema de la tabla para informar al LLM.

In [ ]:
# Connect to database
# We check relative path assuming notebook is in 02-sql-rag/
db_path = Path("data/hybrid_cars.db")
if not db_path.exists():
    # Fallback for absolute path if running from root context
    db_path = Path("02-sql-rag/data/hybrid_cars.db")

print(f"Using database at: {db_path.absolute()}")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get database schema for the LLM
cursor.execute("PRAGMA table_info(cars)")
schema_info = cursor.fetchall()
schema_description = "Table: cars\nColumns:\n"
for col in schema_info:
    schema_description += f"  - {col[1]} ({col[2]}): "
    if col[1] == "vehicle":
        schema_description += "Name of the car model"
    elif col[1] == "year":
        schema_description += "Year of manufacture"
    elif col[1] == "msrp":
        schema_description += "Manufacturer's Suggested Retail Price in dollars"
    elif col[1] == "acceleration":
        schema_description += "0-60 mph time in seconds (lower is faster)"
    elif col[1] == "mpg":
        schema_description += "Miles per gallon (higher is more efficient)"
    elif col[1] == "class":
        schema_description += "Vehicle class (Compact, Midsize, SUV, etc.)"
    schema_description += "\n"

print("\nDatabase Schema:")
print(schema_description)

Definición de los mensajes del sistema (prompts) para la generación de SQL y la respuesta en lenguaje natural.

In [ ]:
# System message for SQL generation
SQL_GENERATION_SYSTEM_MESSAGE = f"""
You are a SQL expert assistant. Your task is to convert natural language questions into valid SQLite queries.

Database schema:
{schema_description}

Important rules:
1. Generate ONLY the SQL query, no explanations or markdown formatting
2. Use proper SQLite syntax
3. Always use SELECT statements (no INSERT, UPDATE, DELETE, DROP)
4. Use appropriate WHERE, ORDER BY, LIMIT clauses as needed
5. For "fastest" questions, use ORDER BY acceleration ASC (lower is faster)
6. For "slowest" questions, use ORDER BY acceleration DESC
7. For "most efficient" or "best mpg", use ORDER BY mpg DESC
8. For "cheapest", use ORDER BY msrp ASC
9. For "most expensive", use ORDER BY msrp DESC
10. If asking for a single result or "the", add LIMIT 1
11. Use LIKE for partial matches (e.g., WHERE vehicle LIKE '%Prius%')
12. Consider conversation context to refine queries

Examples:
Q: "how fast is the prius v?"
A: SELECT vehicle, acceleration, year, mpg FROM cars WHERE vehicle LIKE '%Prius V%'

Q: "what is the fastest car?"
A: SELECT vehicle, acceleration, year, msrp FROM cars ORDER BY acceleration ASC LIMIT 1

Q: "show me efficient cars"
A: SELECT vehicle, mpg, year, class FROM cars WHERE mpg > 40 ORDER BY mpg DESC

Q: "give me the cheapest pickup truck"
A: SELECT vehicle, msrp, year, mpg FROM cars WHERE class = 'Pickup Truck' ORDER BY msrp ASC LIMIT 1
"""

ANSWER_SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based on a hybrid car database.
You must use the data from the query results to answer the questions accurately.
Be concise and direct in your answers. Use the conversation history to provide context-aware responses.
"""

Función principal que orquesta el flujo: pregunta -> SQL -> ejecución -> respuesta natural.

In [ ]:
def process_question(question, chat_history=None):
    if chat_history is None:
        chat_history = [{"role": "system", "content": ANSWER_SYSTEM_MESSAGE}]
    
    print(f"Question: {question}")
    
    try:
        # Generate SQL query using the LLM with conversation context
        sql_response = client.chat.completions.create(
            model=MODEL_NAME,
            temperature=0.1,
            messages=[
                {"role": "system", "content": SQL_GENERATION_SYSTEM_MESSAGE},
                {
                    "role": "user",
                    "content": f"New question: {question}\n\nConversation context: {chat_history[-4:] if len(chat_history) > 4 else chat_history}",
                },
            ],
        )

        sql_query = sql_response.choices[0].message.content.strip()
        # Remove any markdown code blocks if present
        if sql_query.startswith("```"):
            sql_query = sql_query.split("\n", 1)[1]
            sql_query = sql_query.rsplit("```", 1)[0]
        sql_query = sql_query.strip()

        print(f"\nGenerated SQL: {sql_query}")

        # Execute the query
        cursor.execute(sql_query)
        results = cursor.fetchall()

        # Get column names
        column_names = [description[0] for description in cursor.description]

        # Format results as markdown table
        if results:
            results_table = " | ".join(column_names) + "\n"
            results_table += " | ".join("---" for _ in column_names) + "\n"
            for row in results[:10]:  # Limit to 10 rows for display
                results_table += " | ".join(str(val) if val is not None else "N/A" for val in row) + "\n"

            if len(results) > 10:
                results_table += f"\n... and {len(results) - 10} more rows"

            print(f"\nQuery results ({len(results)} rows):")
            print(results_table)
        else:
            results_table = "No results found."
            print(f"\n{results_table}")

        # Use the results to generate a natural language response
        chat_history.append({"role": "user", "content": f"{question}\n\nQuery results:\n{results_table}"})

        response = client.chat.completions.create(
            model=MODEL_NAME, temperature=0.3, messages=chat_history
        )

        llm_response = response.choices[0].message.content
        chat_history.append({"role": "assistant", "content": llm_response})

        print(f"\nResponse:")
        print(llm_response)
        
        return chat_history

    except sqlite3.Error as e:
        print(f"\nSQL Error: {e}")
        print("The generated query might be invalid.")
        return chat_history
    except Exception as e:
        print(f"\nError: {e}")
        return chat_history

Inicialización del historial de conversación y ejecución de ejemplos de preguntas.

In [ ]:
# Initialize conversation history
history = [{"role": "system", "content": ANSWER_SYSTEM_MESSAGE}]

In [ ]:
# Prompt 1
history = process_question("whats the most expensive car for each year", history)

In [ ]:
# Prompt 2
history = process_question("Which car has the best acceleration (lowest 0-60 time)?", history)

In [ ]:
# Prompt 3
history = process_question("Show me the top 3 most fuel efficient SUVs", history)